In [14]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cbook as cbook
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import dense_to_sparse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.metrics.pairwise import haversine_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GridSearchCV, train_test_split
from skorch import NeuralNetRegressor
import itertools
import pickle

# === 1. GCN-LSTM Model ===
class GCN_LSTM_Masked_Time(nn.Module):
    def __init__(self, in_channels, hidden_dim, lstm_hidden_dim, out_dim):
        super().__init__()
        self.gcn = GCNConv(in_channels, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, lstm_hidden_dim, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_dim, out_dim)

    def forward(self, x, edge_index, mask=None):
        B, T, N, feat_dim = x.size()   # renamed from F to feat_dim
        gcn_outputs = []

        for t in range(T):
            x_t = x[:, t, :, :].reshape(B * N, feat_dim)
            gcn_out = self.gcn(x_t, edge_index)
            gcn_out = F.relu(gcn_out)
            gcn_out = gcn_out.view(B, N, -1)
            gcn_outputs.append(gcn_out)

        gcn_outputs = torch.stack(gcn_outputs, dim=1)
        lstm_input = gcn_outputs.permute(0, 2, 1, 3).contiguous().view(B * N, T, -1)
        lstm_out, _ = self.lstm(lstm_input)
        out = self.fc(lstm_out).view(B, N, T, -1)
        return out.permute(0, 2, 1, 3).squeeze(-1)

In [2]:
def split_features_target(data_np, target_index):
    T, N, F = data_np.shape
    assert 0 <= target_index < F
    y = data_np[:, :, target_index]                    # [T, N]
    x = np.delete(data_np, target_index, axis=-1)      # [T, N, F-1]
    return x, y

In [3]:
def build_input_target_rolling(data_np, target_np, seq_len, mask_np=None):
    T, N, F = data_np.shape
    num_windows = T - seq_len + 1
    x, y, mask = [], [], []

    for t in range(num_windows):
        x_t = data_np[t:t+seq_len]
        y_t = target_np[t:t+seq_len]

        if mask_np is not None:
            mask_t = mask_np[t:t+seq_len]
        else:
            mask_t = ~np.isnan(y_t) & ~np.isnan(x_t[:, :, 0])
            mask_t = mask_t.astype(np.float32)

        x.append(np.nan_to_num(x_t, nan=0.0))
        y.append(np.nan_to_num(y_t, nan=0.0))
        mask.append(mask_t)

    return np.stack(x), np.stack(y), np.stack(mask)  # [B, T, N, F], [B, T, N], [B, T, N]

In [4]:
def train_time(model, x, y, mask, edge_index, num_epochs=10, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss(reduction='none')

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        x_t = torch.tensor(x).float()
        y_t = torch.tensor(y).float()
        mask_t = torch.tensor(mask).float()

        pred = model(x_t, edge_index, mask_t)  # [B, T, N]
        loss = loss_fn(pred, y_t) * mask_t
        loss = loss.sum() / mask_t.sum()

        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

In [5]:
def evaluate_mse_time(model, x, y, mask, edge_index, device='cpu'):
    model.eval()
    model.to(device)

    with torch.no_grad():
        x_t = torch.tensor(x).float().to(device)
        y_t = torch.tensor(y).float().to(device)
        mask_t = torch.tensor(mask).float().to(device)

        pred = model(x_t, edge_index, mask_t)
        mse = (((pred - y_t) ** 2) * mask_t).sum() / mask_t.sum()
    return mse.item()

In [6]:
df = pd.read_csv("bc_merged_data/Final_AQS_CSN_2003_2023_BC_Data.csv")

In [7]:
df["one_over_dist"] = 1/df["fire_dist"]
df["timestamp"] = pd.to_datetime(df["Date"])
df = df.sort_values(by=['timestamp', 'EMS_ID'])

features = ['TEMP_MEAN_XB0C_DAILY', 'WDIR_UVEC_DEG_DAILY', "WSPD_VECT_MS_DAILY", "one_over_dist", "Season", "smoke_score", "log_PM25"]
df_pivoted = df.pivot(index='timestamp', columns='EMS_ID', values = features)
df_pivoted.columns = [f'station{s}_{f}' for s, f in df_pivoted.columns]
df_2d = df_pivoted.to_numpy()

In [8]:
stations = pd.Series(list(zip(df['Latitude'].round(3), df['Longitude'].round(3))))
stations = np.array(stations.drop_duplicates().reset_index(drop=True).tolist())
stations_rad = np.radians(stations)
dist_matrix_km = haversine_distances(stations_rad) * 6371

k = 2
nbrs = NearestNeighbors(n_neighbors=k+1, metric='precomputed').fit(dist_matrix_km)
_, indices = nbrs.kneighbors(dist_matrix_km)

edges = []
for i, neighbors in enumerate(indices):
    for j in neighbors[1:]:  # skip self-loop
        edges.append([i, j])
        edges.append([j, i])  # make it undirected

edge_index = torch.tensor(edges).T  # shape: [2, num_edges]

print(edge_index)

tensor([[ 0, 10,  0, 17,  1, 38,  1,  6,  2, 36,  2,  5,  3, 12,  3, 37,  4, 27,
          4, 22,  5, 13,  5, 36,  6, 38,  6,  1,  7, 37,  7, 12,  8,  9,  8, 15,
          9,  8,  9, 15, 10,  0, 10, 17, 11,  3, 11, 12, 12, 37, 12,  7, 13,  5,
         13, 36, 14, 34, 14, 10, 15,  8, 15,  9, 16,  1, 16, 38, 17, 10, 17,  0,
         18, 33, 18, 31, 19, 30, 19, 20, 20, 19, 20, 30, 21, 23, 21, 22, 22, 23,
         22, 21, 23, 22, 23, 21, 24, 32, 24, 39, 25, 26, 25, 31, 26, 25, 26, 31,
         27,  4, 27, 22, 28, 34, 28, 14, 29,  6, 29,  1, 30, 19, 30, 20, 31, 33,
         31, 18, 32, 39, 32, 24, 33, 18, 33, 31, 34, 14, 34, 10, 35,  9, 35,  8,
         36,  5, 36, 13, 37,  7, 37, 12, 38,  1, 38,  6, 39, 32, 39, 24],
        [10,  0, 17,  0, 38,  1,  6,  1, 36,  2,  5,  2, 12,  3, 37,  3, 27,  4,
         22,  4, 13,  5, 36,  5, 38,  6,  1,  6, 37,  7, 12,  7,  9,  8, 15,  8,
          8,  9, 15,  9,  0, 10, 17, 10,  3, 11, 12, 11, 37, 12,  7, 12,  5, 13,
         36, 13, 34, 14, 10, 14,  8

In [9]:
num_stations = len(stations)
num_features = len(features)
time_steps = len(df_pivoted)

df_3d = df_2d.reshape(time_steps, num_stations, num_features)

In [10]:
target_index = 6
x_np, y_np = split_features_target(df_3d, target_index=target_index)

In [15]:
# Prepare tensors
seq_len=30
x, y, mask = build_input_target_rolling(x_np, y_np, seq_len=seq_len)  # target is feature 0

# Instantiate model
model = GCN_LSTM_Masked_Time(
    in_channels=x.shape[-1],
    hidden_dim=16,
    lstm_hidden_dim=32,
    out_dim=1,
)

# Train
train_time(model, x, y, mask, edge_index, num_epochs=10, lr=0.01)
mse = evaluate_mse_time(model, x, y, mask, edge_index)
print("Final MSE:", mse)

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 20444476344 bytes.